## Introduction
- Cricket is a bat-and-ball game played between two teams of eleven players each on a cricket field, at the centre of which is a rectangular 20-metre (22-yard) pitch with a target at each end called the wicket (a set of three wooden stumps upon which two bails sit). Each phase of play is called an innings, during which one team bats, attempting to score as many runs as possible, whilst their opponents bowl and field, attempting to minimise the number of runs scored. When each innings ends, the teams usually swap roles for the next innings (i.e. the team that previously batted will bowl/field, and vice versa). The teams each bat for one or two innings, depending on the type of match. The winning team is the one that scores the most runs, including any extras gained (except when the result is not a win/loss result).
- IPL is a T20 format game in which first 6 overs are batting powerplay where only 2 filders are allowed outside the 30 yard circle.
- Every batting team wants a momentum in these 6 overs.
- Player who score maximum runs in ipl won orange cap
- Player who take maximum wickets in ipl won purple cap

## Data source
- https://www.kaggle.com/manasgarg/ipl

## Tasks performed in this analysis
- Find the toss winning count per team.
- Display the team that wins the most number of matches per season.
- Venue that hosted the most number of matches
- Find the most valuable player.
- The most successful IPL team.
- Which umpire has officiated the most number of IPL matches on-field?
- Does winning the toss has any advantage?
- Is it better to defend or chase in IPL?
- First and second inning batting average per venue.
- Runs scored in a powerplay of every season.
- Most runs scored by a batsmen in every season of IPL.
- Maximum wicket taking bowler in every season of ipl.
- Highest partnership in runs of each season.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def read_datasets():
    """
    Reads the dataset for teams and ball by ball
    
    Args: none
    returns: Pandas DF (data and inning_dataset)
    """
    data = pd.read_csv("ipl-data/IPL Matches 2008-2020.csv")
    data_innings = pd.read_csv('ipl-data/IPL Ball-by-Ball 2008-2020.csv')
    
    data = data.replace(['Rising Pune Supergiants'],'Rising Pune Supergiant')
    data['season'] = pd.DatetimeIndex(data.date).year
    data_innings = data_innings.replace(['Rising Pune Supergiants'],'Rising Pune Supergiant')
    
    return data, data_innings

data, dataset_innings = read_datasets()

## Find the toss winning count per team.
- 'toss_winner' - toss winner teams
- 'id' - per match unique id

In [ ]:
# call 3 columns from data set
toss_winner_team = data.filter(['id','toss_winner','winner'])
#checing null values 
toss_winner_team.isnull().sum()
#drop null values
toss_winner_team.dropna(inplace=True)
#groupby function split the data into groups
toss_winning_count = pd.DataFrame(toss_winner_team.groupby(['toss_winner']).id.count())
#reset the index to the default integer index beginning at 0
toss_winning_count.reset_index(level=0,inplace = True)
#change figure size
plt.figure(figsize=(10,7))
# Bar graph is used to compare different teams
plt.barh(toss_winning_count['toss_winner'],toss_winning_count['id'],edgecolor='black')
plt.title('Toss winning count per Team',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel('Toss Count',fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel('Teams',fontdict= {'fontweight':'bold','fontsize': 14})
plt.show()

### Observation - 
- Bar graph represent the number of toss win by each team from 2008 to 2020.
- X-axis represent the team.
- Y-axis represent the toss winning count.
- There are total 15 teams in competition.
- Mumbai-indians wins the maximum number of tosses followed by kolkata knight riders and chennai super kings.
- Kochi Tuskers Kerala wins minimun number of tosses.

## Display the teams that wins the most number of matches per season.
- season given in a column 'date.
- winning teams

In [ ]:
#call 2 columns from data set
most_win = data.filter(['date','winner'])
#create a new column 'season' on bases of year
most_win['season'] = pd.DatetimeIndex(most_win['date']).year
#groupby function split the data into groups
win_season = most_win.groupby(['season','winner']).agg(wins=('winner', 'count'))
#create a new column,rank values on the bases of wins
win_season['rank_wins'] = win_season.groupby('season')['wins'].rank(method='first',ascending=False).astype(np.int32)
win_season.reset_index(level = 0,inplace=True)
#condition - return 1st rank values
win_year = win_season[win_season.rank_wins==1]
win_year.reset_index(level = 0,inplace=True)
fig = px.scatter(win_year, x="winner", y="wins", color="season",
                 size='wins',title='Teams wins most number of matches per season')
fig.update_xaxes(title_text="Teams")
fig.update_yaxes(title_text="Total number of wins")
fig.show()

### Observation
- X-axis represent teams.
- Y-axis represent number of wins.
- Total teams 7. 
- Mumbai Indians wins most number of matches in 3 seasons 2013, 2017 and 2020. They won maximum 13 matches in 2013.
- Chennai Super Kings wins in 2 seasons 2015 and 2018. They won maximum 11 matches in 2018.

## Venue that hosted the most number of matches
- 'venue' - playground 
- number of matches define on the bases of id.

In [ ]:
#groupby function split the data into groups
venue_max = pd.DataFrame(data.groupby(['venue']).id.count())
#reset the index to the default integer index beginning at 0
venue_max.reset_index(level=0,inplace=True)
#sort the values of id in decending order
high_low = venue_max.sort_values(by='id',ascending=False)
#changing figure size
plt.figure(figsize=(10,7))
# Bar graph is used to compare different grounds
plt.bar(venue_max['venue'],venue_max['id'],edgecolor='black')
plt.title('Number of matches played on each ground',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel('Venue',fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel('Number of matches',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xticks(rotation=-90)
plt.show()

### Observation - 
- Bar graph represent the number of matches played on each ground.
- X-axis represent playgrounds name.
- Y-axis represent number of matches.
- From 2008 to 2020 matches were hosted by 36 venues.
- Maximum number of matches played on Eden Gardens followed by Feroz Shah Kotla and Wankhede Stadium.

## Find the most valuable player.
- player of the match

Limitation
- Valuable player is not available in a dataset.

Assumption 
- The player who won most player of the match is the most valuable player. 

In [ ]:
#call player_of_match column from dataset
val_player = data.filter(['player_of_match'])
#groupby function split the data into groups
#create new column and count the number of player occur each time
most_val_count = val_player.groupby(['player_of_match']).agg(man_of_match=('player_of_match','count'))
#reset index to default
most_val_count.reset_index(level=0,inplace=True)
arrange_high_low = most_val_count.sort_values(by='man_of_match',ascending=False)
print(arrange_high_low.shape)
plt.figure(figsize=(10,7))
plt.bar(arrange_high_low['player_of_match'][0:10],arrange_high_low['man_of_match'][0:10],edgecolor='black')
plt.title('Top 10 Valuable Player',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel('Players Name',fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel('Award count',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xticks(rotation=-45)
plt.show()

### Observation - 
- Bar graph represent the top 10 valuable players.
- X-axis represent players name.
- Y-axis represent number of times award winning by each player.
- Most valuable player is AB-de villiers.He won 23 times player of the match.
- There are 6 indian player in top 10 valuable player.

## Alternate method to create a dataset.

In [ ]:
#count each name from a column
player = data['player_of_match'].value_counts()
#create 2 new column
match_player = pd.DataFrame(columns=['player_name','player_match_count'])
match_player.dtypes
#iterating values by using for loop
# iteritems returns an iterator of the dictionary’s list in the form of (key, value) tuple pairs
for items in player.iteritems():
# add values in column by indexing
    add_value_columns = pd.DataFrame({'player_name':[items[0]],'player_match_count':[items[1]]})
    match_player = match_player.append(add_value_columns,ignore_index=True)
match_player.head()

## The most successful IPL team.
- winner teams.

Limitation
- There is no particular defination of a successful IPL team.

Assumption
- The team that has registered the most number of victories is the most successful IPL team.

In [ ]:
#call winner column from dataset
team_win = data.filter(['winner'])
#groupby function split the data into groups
#create new column and count the number of team wins
teams_winning_count = team_win.groupby(['winner']).agg(win_count=('winner','count'))
#reset index to default
teams_winning_count.reset_index(level=0,inplace=True)
plt.figure(figsize=(10,7))
plt.barh(teams_winning_count.winner,teams_winning_count.win_count,edgecolor='black')
plt.title("Teams winning count",fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel('Winning count',fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel('Teams',fontdict= {'fontweight':'bold','fontsize': 14})
plt.show()

### Observation - 
- Bar graph repesent the total number of wins by each team from 2008 to 2020.
- X-axis represent teams .
- Y-axis represent total number of wins.
- Maximum number of matches wins by mumbai indians followed by chennai super kings and kolkata knight riders.

## Which umpire has officiated the most number of IPL matches on-field?  
- umpire1 and umpire2.

In [ ]:
#read column from data
first_umpire = data.filter(['umpire1'])
#check data type
first_umpire.dtypes 
#check null values
first_umpire.isnull().sum()
#read column from data
second_umpire = data.filter(['umpire2'])
#check data type
first_umpire.dtypes
#check null values
second_umpire.isnull().sum()
#two dataframes stored in new variable
frames = [first_umpire,second_umpire]
#concatenate 2 columns in a single column
total_umpire = pd.concat(frames)
total_umpire.shape
#check duplicate values
total_umpire.umpire1.duplicated().sum()
#groupby function split the data into groups,create new column and count umpire's IPL matches on-field
count_umpire = pd.DataFrame(total_umpire.groupby(['umpire1']).agg(umpire_on_field=('umpire1','count')))
#create a new column and using rank function 
count_umpire['rank_umpire'] = count_umpire.umpire_on_field.rank(method = 'first',ascending = False).astype(np.int32)
#condition read top 10 values
count_umpire = count_umpire[count_umpire.rank_umpire < 11]
#reset index on default
count_umpire.reset_index(level=0,inplace = True)
plt.figure(figsize=(10,7))
plt.barh(count_umpire['umpire1'],count_umpire['umpire_on_field'])
plt.title("Umpire's officiated IPL matches",fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel("Number of matches",fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel("Umpire's Name",fontdict= {'fontweight':'bold','fontsize': 14})
#plt.xticks(rotation=90)
plt.show()

### Observation - 
- Bar graph represent top 10 umpires officiated maximum number of ipl matches.
- X-axis represent total number of matches.
- Y-axis represent umpires name.
- HDPK Dharmasena has officiated most number of ipl matches followed by AK chaudhary.

## Does winning the toss has any advantage? 
- Toss winner teams
- Match winner teams

In [ ]:
#read a column from data
toss_winning_teams = data.filter(['toss_winner'])
#split data into a group by using groupby function
#make a new column and count toss win per team
toss_winning_count = pd.DataFrame(toss_winning_teams.groupby(['toss_winner']).agg(toss_count=('toss_winner','count')))
#reset index to default
toss_winning_count.reset_index(level=0,inplace=True)
#read 2 columns from data
toss_match_winner = data.filter(['toss_winner','winner'])
#winning the toss also wins the match
winner = toss_match_winner[toss_match_winner['toss_winner']==toss_match_winner['winner']]
#group toss winner data and count winner team in a new column
toss_match_win = pd.DataFrame(winner.groupby(['toss_winner']).agg(match_win_count=('winner','count')))
toss_match_win.reset_index(level=0,inplace=True)
#change column names
toss_match_win.rename(columns={'toss_winner' : 'Teams','match_win_count':'Toss/Match win'},inplace=True)
#2 data frame in a variable
frames1 = [toss_match_win,toss_winning_count]
#merge 2 dataset
result = pd.concat(frames1,axis=1)
#change column name
result.rename(columns={'toss_count':'Total toss win'},inplace=True)
#drop unnecesary column
result.drop(columns=['toss_winner'],inplace=True)
result['Toss win percentage'] = (result['Toss/Match win'] / result['Total toss win']) * 100
plt.figure(figsize=(10,7))
plt.barh(result['Teams'],result['Toss win percentage'])
plt.title("Match winning percentage",fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel("Winning percentage",fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel("Teams",fontdict= {'fontweight':'bold','fontsize': 14})
plt.show

## Is it better to defend or chase in IPL?
- winner
- result
- result margin

In [ ]:
defend_chase = data.filter(['winner','result','result_margin'])
#define a function
def defend(x):
#return teams win defend    
    if (x['result'] == 'runs'):
        return x['winner']
    else:
        return None
def chase(y):
    if (y['result'] == 'wickets'):
        return y['winner']
    else:
        return None
defend_chase['defending_team'] = defend_chase.apply(defend,axis=1)
defend_chase['chasing_team'] = defend_chase.apply(chase,axis=1)
defend = pd.DataFrame(defend_chase.groupby(['defending_team']).agg(defend_count=('defending_team','count')))
defend.reset_index(level=0,inplace=True)
defend.head()
chase = pd.DataFrame(defend_chase.groupby(['chasing_team']).agg(chasing_count=('chasing_team','count')))
chase.reset_index(level=0,inplace=True)
chase.head()
fig = go.Figure(data=[
    go.Bar(name='Chase', x=chase.chasing_team, y=chase.chasing_count),
    go.Bar(name='Defend', x=defend.defending_team, y=defend.defend_count)
])
fig.update_layout(barmode='group',title='Win by chase/defend the score')
fig.update_xaxes(title_text="Teams")
fig.update_yaxes(title_text="Number of wins")
fig.show()

## Observation
- X-axis represent teams.
- Y-axis represent total number of wins.
- Chennai Super Kings and Pune warriors are the teams whose winning count is same in both the condition. But Chennai Super Kings played more matches than Pune warriors.
- Deccan Chargers, Delhi Capitals and Sunrisers Hyderabad these three teams won more number of matches in defending the score.
- Kolkata Knight Riders won maximum number of matches by chasing the score.

## First and second inning batting average per venue.

In [ ]:
agg_innings = dataset_innings.groupby(['id', 'inning', 'batting_team']).agg({'total_runs': 'sum'})
agg_innings.reset_index(level=[0,1,2], inplace=True)
new_dataset = data.merge(agg_innings, on = 'id')
temp_dataset = new_dataset.filter(['id', 'venue','inning', 'batting_team', 'total_runs'])
venue_data = pd.DataFrame(temp_dataset.groupby(['venue', 'inning']).total_runs.mean())
venue_data.reset_index(level=[0,1], inplace=True)
first_inning = venue_data[venue_data.inning == 1]
second_inning = venue_data[venue_data.inning == 2]
fig = go.Figure(data=[
    go.Bar(name='First inning', x=first_inning.venue, y=first_inning.total_runs),
    go.Bar(name='secong inning', x=second_inning.venue, y=second_inning.total_runs)
])
fig.update_layout(barmode='group',title='1st and 2nd inning average runs',width=800,height=800)
fig.update_xaxes(title_text="Venue")
fig.update_yaxes(title_text="Average runs")
fig.show()

## Runs scored in a powerplay of every season.
- match id
- overs
- total runs
- date
- batting team

In [ ]:
season = data.filter(['date','id'])
season['season'] = pd.DatetimeIndex(data['date']).year
season.drop(columns='date',inplace=True)
#checking null values
season.isnull().sum()
innings = dataset_innings.filter(['id','over','total_runs','batting_team'])
innings.isnull().sum()
new_data = season.merge(innings, on = 'id')
new_data.shape
powerplay = new_data[new_data['over'] < 6] 
new_powerplay = pd.DataFrame(powerplay.groupby(['batting_team','season']).agg({'total_runs':'sum'}))
new_powerplay.reset_index(level=[0,1],inplace=True) 
# new_powerplay = new_powerplay[new_powerplay['season'] == 2008]
# fig = px.bar(new_powerplay, x='batting_team', y='total_runs', 
#              color='season',labels={'color':'Season'}, height=500)
# fig.update_xaxes(title_text="Teams")
# fig.update_yaxes(title_text="Runs")
# fig.show()
fig = px.scatter(new_powerplay, x='batting_team', y='total_runs', 
             color='season',labels={'color':'Season'}, height=500)
fig.update_layout(title='Powerplay runs')
fig.update_xaxes(title_text="Teams")
fig.update_yaxes(title_text="Runs")
fig.show()

## Most runs scored by a batsmen in every season of IPL.
- Batsmen
- season
- Batsmen runs

In [ ]:
score_data = dataset_innings.filter(['id','batsman','batsman_runs'])
runs_new_data = season.merge(score_data, on = 'id')
# runs_new_data.drop(columns='date',inplace=True)
most_runs = pd.DataFrame(runs_new_data.groupby(['season','batsman']).agg({'batsman_runs':'sum'}))
most_runs.reset_index(level=[0,1],inplace=True)
most_runs['rank_batsman'] = most_runs.groupby('season')['batsman_runs'].rank(method='first',ascending=False).astype(np.int32)
# most_runs = most_runs[most_runs['batsman'] == 'V Kohli']
most_runs = most_runs[most_runs.rank_batsman==1]
plt.subplots(figsize=(15,10))
sns.barplot(x='batsman_runs',y='batsman',data=most_runs,hue='season')
plt.title('Orange cap holders',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel('Batsman runs',fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel('Batsman name',fontdict= {'fontweight':'bold','fontsize': 14})
plt.show()

### Observation
- X-axis represent runs scored by batsmen.
- Y-axis represent batsmen.
- CH Gayle 2 times orange cap winner in year 2011,2012 and he scored more runs in 2012 than 2011.
- DA Warner won orange cap in three consecutive years 2015, 2016 and 2017.In all three years he scored more runs in 2017.
- As we compare all the seasons V Kohli scored maximum runs in year 2018 and won orange cap . 

## Maximum wicket taking bowler in every season of ipl.
- Bowler
- season
- wicket

In [ ]:
wicket = dataset_innings.filter(['id','bowler','is_wicket'])
wicket_new_data = season.merge(wicket, on = 'id')
max_wicket = pd.DataFrame(wicket_new_data.groupby(['season','bowler']).agg({'is_wicket':'sum'}))
max_wicket.reset_index(level=[0,1],inplace=True)
max_wicket['rank_bowler'] = max_wicket.groupby('season')['is_wicket'].rank(method='first',ascending=False).astype(np.int32)
max_wicket = max_wicket[max_wicket.rank_bowler==1]
plt.subplots(figsize=(15,10))
sns.scatterplot(x='season',y='is_wicket',data=max_wicket,hue='bowler',s=150,palette='muted')
plt.title('Purple cap holders',fontdict= {'fontweight':'bold','fontsize': 14})
plt.xlabel('Season',fontdict= {'fontweight':'bold','fontsize': 14})
plt.ylabel('Wickets',fontdict= {'fontweight':'bold','fontsize': 14})
plt.show()

### Observation -
- X-axis represent the season of ipl.
- Y-axis represent the number of wickets.
- Sohail tanvir was the first bowler awarded purple cap.
- Dj bravo,B kumar and K-Rabada was 2 times purple cap winners.
- Dj bravo takes maximum 34 wickets in year 2013

## Highest partnership in runs of each season.
- 'id' - particular match have an unique match id.
- 'batting_team' - team batting in a match. 
- 'batsman' - who are facing the ball.
- 'non-striker' - batsman on the other side of crease.
- 'total_runs' - batsman runs including extras runs like no-ball,wide etc.

In [ ]:
data, dataset_innings = read_datasets()
max_patnership = pd.DataFrame(dataset_innings.groupby(['id','inning','batsman','non_striker','batting_team']).agg({'total_runs':'sum'}))
max_patnership['rank_runs'] = max_patnership.groupby(['id'])['total_runs'].rank(method='first',ascending=False).astype(np.int32)
max_patnership = max_patnership[max_patnership.rank_runs==1]
max_patnership.reset_index(level=[0,1,2,3,4],inplace=True)
season = data.filter(['id', 'season'])
partnership_season = season.merge(max_patnership, on = 'id')
partnership_season['rank_season'] = partnership_season.groupby(['season'])['total_runs'].rank(method='first',ascending=False).astype(np.int32)
partnership_season = partnership_season[partnership_season.rank_season==1]
partnership_season.drop(columns=['rank_runs', 'rank_season'], inplace=True)
fig = px.bar(x=partnership_season.season, y=partnership_season.total_runs,
            color=partnership_season.batting_team, facet_row=partnership_season.inning,labels={'x':'Years','y':'Runs','color':'Teams'},
            title='Maximum runs partnership per year')
fig.show()

### Observation
- X-axis represent years.
- Y-axis represent runs.
- Royal Challengers Banglore scored 4 times the maximum runs patnership in year 2012, 2013, 2015 and 2016. Out of 4 they scored maximum 138 runs in year 2015.
- Sunrisers Hyderabad scored maximum partnership in 3 consecutive years 2017 to 2019. Out of 3 they scored maximum 115 runs in year 2019.
- Kings XI Punjab also scored in 3 years 2011, 2014 and 2020. Out of three they score maximum 126 runs in year 2011.
- In year 2009 and 2010 Rajasthan Royals scored maximum runs partnership. They scored maximum 89 runs in 2010.